## importing required libraries

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from collections import defaultdict
import struct
import numpy as np
import numpy
import random
import tensorflow as tf
import os
import pickle as pickle
import matplotlib.pyplot as plt
import sys
from datetime import datetime
import time
from pathlib import Path
p = !pwd
home = p[0].replace('/notebook','')
sys.path.insert(0, os.path.join(home,'src','model'))
print(os.path.join(home,'src','model'))
from bpr_model import *
%matplotlib inline
print(home)

/Users/nolanthomas/Documents/capstone/bpr_project/src/model


SyntaxError: invalid syntax (bpr_model.py, line 375)

## Load and make transformations; reuse across sessions
#### 1. Load data
#### 2. Make transformations
#### 3. Run Session
#### 4. Save Results dictionary into a pickle 
#### 5. Pickle files can be review later for comparison/combining/graphing/performance analysis

In [ ]:
bpr_data_file = os.path.join(home, 'data', 'processed', 'out_model_features.20180523.tab.csv')

In [2]:
user_count, item_count, users, items, user_ratings, item_ratings, loaded_features   = load_data_hybrid(bpr_data_file, 
                                                                                                       min_items=1, 
                                                                                                       min_users=1)
# len(take(1, loaded_features['brand'].values())[0])

NameError: name 'load_data_hybrid' is not defined

In [ ]:
len(loaded_features)

In [ ]:
"""
When: min_items=4, min_users=0

Then:
max_u_id:  34107
max_i_id:  10318
reviews :  131979
u_id:  10244
i_id:  10318
reviews :  69390
"""

In [ ]:
loaded_features = transform_features(loaded_features)
results = {}

In [ ]:
#loaded_features['brand']

## Model Running given different variants

In [ ]:
# define number of sessions or epochs
NUM_SESSIONS = 1
variants  = {
    'BPR':None
    #'Cluster':['cluster_1hotencoded']
    #'Price-10bin':['price_1hotencoded'],
    #'Price-Log-30bin':['price_log_1hotencoded'],
    #'L4-Avg-10bin':['level4_average_1hotencoded'],
    #'L4-Avg-Log-30bin':['level4_average_log_1hotencoded']
    #'Price-Adj-10bin':['l4_avg_adjusted_1hotencoded'],
    #'Price-Adj-Log-30bin':['l4_avg_adjusted_log_1hotencoded']
    #'Category-Tags, Subcat-L4, Brand':['top_categories','level4', 'brand_1hotencoded']
    #'Category-Tags':['top_categories'],
    #'Subcat-L4':['level4'],
    #'Price-Log':['price_log_1hotencoded'],
    #'Price-Adj':['l4_avg_adjusted_1hotencoded'],
    #'Price-Adj-Log':['l4_avg_adjusted_log_1hotencoded'],
    #'Brand':['brand_1hotencoded']
}
"""
variants  = {
    #'Price-Log':['price_log_1hotencoded'],
    'Category-Tags, Subcat-L4, $Price':['top_categories','level4','price_log_1hotencoded'],
    'Subcat-L4, Price-Log':['level4','price_log_1hotencoded'],
    'Category-Tags, Price-Log':['top_categories','price_log_1hotencoded'],
    'Category-Tags, Subcat-L4, Brand, Price-Log':['price_log_1hotencoded','top_categories', 'level4', 'brand_1hotencoded']
}
"""

In [ ]:
# feature_set() is called without arguments
for desc, features_to_use in variants.items():
    print (desc + str(features_to_use))
    if features_to_use != None:
        features_list = feature_set([loaded_features[c] for c in features_to_use])
    else:
        features_list = feature_set()
    results[desc] = run(1,
                        user_count, item_count,
                        users, items,
                        user_ratings, item_ratings,
                        features_list,
                        hidden_dim=10,hidden_adv_dim=10,
                        cold_start_thresh=10
                       )

In [ ]:
results.keys()

### Save results dictionary to a pickle file

In [ ]:
import pickle
datetime.now().strftime("%Y%m%d.%H%M")
pickle.dump( results, open( "results."+datetime.now().strftime("%Y%m%d.%H%M")
 + ".pickle", "wb" )

## Single Features

In [ ]:
#single_features = ['BPR','Category-Tags','Subcat-L4','Price-Log','Price-Adj-Log','Brand']
#results_to_graph ={v:results[v] for v in single_features}
#'Price-10bin', 'Price-Log-30bin', 'L4-Avg-10bin', 'Price-Adj-10bin', 'Price-Adj-Log-30bin']
results_to_graph ={v:results[v] for v in single_features}

In [ ]:
rpt_date = datetime.now().strftime("%Y%m%d.%H%M")
plot_auc_curve(results_to_graph,'BPR Price Features Test AUC\nK=10,Min Items=4,Min Users=0\n' + rpt_date ,None)

plot_auc_cold_start_curve(results, 'BPR Price Features Cold Start AUC\nK=10,Min Items=4,Min Users=0\n' + rpt_date,None)

### Run Feature Combinations: Comparing to BPR and Category-Tags

In [ ]:
combos = ['BPR','Category-Tags', 'Category-Tags, Subcat-L4, Brand']
results_to_graph ={v:results[v] for v in combos}
plot_auc_curve(results_to_graph,
               'BPR Test AUC\nCategory Tags vs Feature Combinations\nK=10,Min Items=4,Min Users=0\n' + rpt_date,
               'Category-Tags')

In [ ]:
plot_auc_cold_start_curve(results_to_graph,
                          'BPR Cold Start Test AUC\nCategory Tags vs Feature Combinations\nK=10,Min Items=4,Min Users=0\n' + rpt_date,
                          'Category-Tags')

### Load results from prior runs for analysis and/or merging

In [ ]:
import pickle
results_0141 = pickle.load( open( "results.20180511.0141.pickle", "rb" ) )

In [ ]:
z = [v['delta_sec'] for k,v in results.items()]

In [ ]:
dt_str = datetime.now().strftime("%Y%m%d.%H%M")
sns.set_context("talk")
plt.figure(figsize=(20,10))
plt.tick_params(axis='both', which='major', labelsize=20)
#plt.title(title,fontsize=30)
plt.scatter(z,
    #label=calc_desc,
    #linewidth=lw,
    #linestyle=ls,
    #markersize=ms,
    marker='o')
plt.legend()
#plt.ylabel("Cold Start Test AUC",fontsize=20)
#plt.xlabel("Number of Iterations",fontsize=20)

### Simple output of AUC value from last iteration

In [ ]:
for k,v in results.items():   
    print('AUC Test -' + k +  str(v['auc_test'][-1]))
    print('Cold Start AUC Test - '+ k + str(v['auc_cold_test'][-1]))    

In [ ]:
## FOR NOLAN TO ADD
user_items_test = results['Category-Tags, Subcat-L4, Brand']['user_items_test']
#def build_recommendation_list(users, items, user_items_test):
f = {}    
for i in (x[0]for x in enumerate(user_items_test)):
    for x in zip(user_items_test[i][0], user_items_test[i][1]):
        f[(x[0][0], x[0][1], x[0][2])] = x[1][0]

user_item_rec_score = {}
user_item_pur_score= {}
for x in list(f.keys()):
    # if SCORE is less than zero meaning USER preffered j Over i
    if f[x] < 0:
        if x[0] not in user_item_rec_score.keys():
            user_item_rec_score[x[0]] = []
        user_item_rec_score[x[0]].append((x[2], f[x]))
    # SCORE of more than 0, meaning user preffered i over j
    else:
        if x[0] not in user_item_pur_score.keys():
            user_item_pur_score[x[0]] = []
        user_item_pur_score[x[0]].append((x[1], f[x]))

#reverse dict for look up
users_lookup= {v:k for k,v in users.items()}
items_lookup= {v:k for k,v in items.items()}

array=[]
for x in list(user_item_rec_score.keys()):
    try:
        a=user_item_rec_score[x]
        b= sorted(a, key=lambda x: x[1])
        #old loop: for i in range(10): 
        #recommending up to top 10 items
        for i in range( len(b) ): #min((len(b), 10)) ):
                array.append((users_lookup[x], items_lookup[b[i][0]]))  
    except IndexError:
        raise IndexError("Index Error at x,i",x,i)


#create dataframe of user and their REC LIST- Top 10 SORTED by high to low Preference score so ORDER matters
df=pd.DataFrame(array, columns=['User','Recommended Items'])
df_Rec=df.groupby('User', as_index=False)['Recommended Items'].agg({'Recommendation list':(lambda x: list(x))})

c=df_Rec.set_index('User').T.to_json()

import json as json
#user_items_test = rz['Category-Tags']['user_items_test']
#rec_json = build_recommendation_list(users, items, user_items_test)

In [ ]:
df_Rec.head()

In [ ]:
print (rec_df.shape, len(user_items_test), len(users), len(items))
df_row =  df_Rec.iloc[574]
print (df_row['User'], df_row['Recommendation list'])

with open('reclist_json', 'w+') as f: 
    f.write(json.dumps(c))

In [ ]:
#!ls *.pickle
import pickle
results = pickle.load( open( "results_k10_minitem1_minuser1.20180521.2306.pickle", "rb" ) )
results['Price-Log'] = results['Price-Log_10']
del results['Price-Log_10']
results.keys()
rpt_date = datetime.now().strftime("%Y%m%d.%H%M")

In [ ]:
plot_auc_curve(results,
               'BPR Test AUC\nK=10,Min Items=1,Min Users=1\n' + rpt_date,
               'Category-Tags')
plot_auc_cold_start_curve(results,
               'BPR Cold Test AUC\nK=10,Min Items=1,Min Users=1\n' + rpt_date,
               'Category-Tags')